In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
@assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
Pkg.activate(;temp=true)
Pkg.add("Revise")
import Revise

Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

pkgs = String[
    "ProgressMeter"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

In [ ]:
PROJECT_BASEDIR = dirname(pwd())
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))

In [ ]:
# fasta_files = sort(filter(x -> occursin(r"\.fna$", x) && !occursin(".vcf", x) && !occursin("normalized", x), readdir(genome_dir, join=true)), by=x->filesize(x))

In [ ]:
short_read_sets = unique(replace.(filter(x -> occursin(r"\.fna\.art", x) && !occursin("minimap2", x) && !occursin("_val_", x) && !isdir(x), sort(readdir(genome_dir, join=true), by=x->filesize(x))), r"\.[12]\.fq.*" => ""))

In [ ]:
function trim_and_filter_short_reads(;forward, reverse, outdir, threads = min(8, Sys.CPU_THREADS))
    Mycelia.add_bioconda_env("trim-galore")
    run(`$(Mycelia.CONDA_RUNNER) run -n trim-galore trim_galore --suppress_warn --cores $(threads) --output_dir $(outdir) --paired $(forward) $(reverse)`)
end

for short_read_set in short_read_sets
    forward = short_read_set * ".1.fq.gz"
    reverse = short_read_set * ".2.fq.gz"
    outdir = dirname(short_read_set)
    # @show outdir
    # x = Mycelia.find_matching_prefix(forward, reverse)÷
    # @show x
    outforward = replace(forward, ".fq.gz" => "_val_1.fq.gz")
    outreverse = replace(reverse, ".fq.gz" => "_val_2.fq.gz")
    if !(isfile(outforward) && isfile(outreverse))
        try
            trim_and_filter_short_reads(forward=forward, reverse=reverse, outdir=outdir)
        catch e
            # no reads for 100bp genome??
            display(e)
        end
    end
end